<a href="https://colab.research.google.com/github/nghoanglong/NLP-Sentiment-Analysis/blob/master/Preprocess_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirements
reader: PATH_FOLDER, split_type=['train', 'test', 'dev']
    
   + return: matrix
    
   + với matrix: row - samples dạng sentence
    
embedding: sentences to id

   + sample sentence return array of indices [2, 4, 6, 6]

## Knowledges
+ Word Embedding 
    + frequency-based
        + one-hot vector
        + tf-idf
        + co-occurence matrix
    + prediction-based
        + CBOW
        + Skip-gram
+ Dense vector embedding -> neural net
+ NLTK xử lý sentence

In [ ]:
import numpy as np
import os
import nltk
nltk.download('punkt')
from nltk.tree import Tree
from nltk.tokenize import word_tokenize
import torch
import torch.nn as nn

In [ ]:
from google.colab import files 
uploaded = files.upload()

In [ ]:
class PreprocessData:
    def __init__(self,
                 FOLDER_PATH):
        """Constructor với tham số nhận vào làm FOLDER_PATH

            FOLDER_PATH định dạng ví dụ: dir_current/data_set/
        """
        self.folder_path = FOLDER_PATH
        self.dataset = None
        self.lib_tokens = None

    def load_dataset(self,
                     type_dataset,
                     file_extension='.txt'):
        """Load dataset tương ứng theo type_dataset

            type_dataset = [train, dev, test] | type = string
            file_extension = .txt, .csv,... | type = string

            return ndarray shape(n, 2) với
                row = np.array[list_tokens extract from sentence, label]
        """
        # try:
        #     DATASET_REQ_PATH = self.folder_path + '/' + type_dataset + file_extension
        #     check_exist = os.path.isfile(DATASET_REQ_PATH)
        #     if check_exist:
        DATASET_REQ_PATH = 'train.txt'
        with open(DATASET_REQ_PATH, 'r') as reader:
            self.dataset = np.array([
                                    np.array([self.PTB_tokenize(line.rstrip("\n")),
                                              self.PTB_get_label(line)], dtype=object)
                                    for line in reader])
        return self.dataset
        #     else:
        #         raise FileExistsError('File nay ko ton tai')
        # except FileExistsError as err:
        #     print(err)
        #     return None

    def PTB_get_label(self,
                      treebank):
        """get label của root sentece trong PTB
            treebank - type string
            return label
        """
        tree = Tree.fromstring(treebank)
        return tree.label()

    def PTB_tokenize(self,
                     treebank):
        """Split list các token từ cây PTB
            
            treebank - type string
            return array = [token, token, token,...]
        """
        tree = Tree.fromstring(str(treebank))
        return np.array(tree.leaves())

    def transfrom_sentence(self,
                           li_tokens):
        """Transfrom list các tokens thành 1 sentence hoàn chỉnh

            li_tokens = [token, token, token,...]
            return sentence
        """
        sentence = ' '.join(li_tokens)
        return sentence

    def get_list_vocabularies(self):
        """Tạo một kho các tokens từ list các sentences

            return dictionary{token: ids, token: ids,...}
        """
        li_tokens = set()
        for sample in self.dataset:
            li_tokens.update(sample[0])
        lib_tokens = dict([(token, idx)
                           for idx, token in enumerate(li_tokens)])
        return lib_tokens

    def encode_sentence(self,
                        sent_tokenized,
                        li_vocabs):
        """Encode một sentence về dạng mỗi token tương ứng với một id trong list vocabs

            sent_tokenized - sentence đã được tokenize thành list các tokens
            li_vocabs = {token: id, token: id,...}
            return sentence = [id, id, id, id,...]
        """

        res_encode = np.array([li_vocabs[token] if token in li_vocabs
                               else print('does not have token in li vocabs')
                               for token in sent_tokenized])
        return res_encode

    def decode_sentence(self,
                        li_vocabs):
        pass


In [ ]:
data = PreprocessData('./data/trees')
train_data = data.load_dataset('train', '.txt')

li_vocabs = data.get_list_vocabularies() # lấy ra list các vocabs
encode_li_sent = np.array([torch.tensor(data.encode_sentence(sample, li_vocabs))
                            for sample in train_data[:, 0]],
                          dtype=object) # encode list các sentence
print(encode_li_sent.shape)